In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras import regularizers
import os
import sklearn.metrics
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
import keras as ks


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/merged_data.csv', index_col='ts', parse_dates=True)

In [ ]:
# Class weights calculation
neg, pos = np.bincount(dataset['anomaly_label'])
total = neg + pos
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
adjusted_weight_for_0 = weight_for_0 * 1.0  # Example: decrease if too many false positives
adjusted_weight_for_1 = weight_for_1 * 1.0  # Example: increase if too many false negatives

class_weight = {0: adjusted_weight_for_0, 1: adjusted_weight_for_1}
time_steps = 50  # Adjust based on your sequence length
batch_size = 128 # Adjust based on your batch size preference
output_size = 1

In [ ]:
# Split data function
def split_data_chronologically(data, test_ratio=0.2, val_ratio=0.2, target_column='anomaly_label'):
    total_samples = len(data)
    test_split_idx = int(total_samples * (1 - test_ratio))
    val_split_idx = int(test_split_idx * (1 - val_ratio / (1 - test_ratio)))

    train_data = data.iloc[:val_split_idx]
    val_data = data.iloc[val_split_idx:test_split_idx]
    test_data = data.iloc[test_split_idx:]

    return train_data, val_data, test_data

# Scale data function
def scale_data(train_data, val_data, test_data, target_column='anomaly_label'):
    scaler = StandardScaler()

    # Fit on training data
    train_features = train_data.drop(columns=[target_column])
    scaler.fit(train_features)

    # Transform features
    train_scaled = scaler.transform(train_features)
    val_scaled = scaler.transform(val_data.drop(columns=[target_column]))
    test_scaled = scaler.transform(test_data.drop(columns=[target_column]))

    # Extract labels
    train_labels = train_data[target_column].values
    val_labels = val_data[target_column].values
    test_labels = test_data[target_column].values

    return train_scaled, train_labels, val_scaled, val_labels, test_scaled, test_labels

# Create TensorFlow dataset function
def create_tf_dataset(features, labels, time_steps=100, batch_size=32, shuffle=False):
    dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=features,
        targets=np.roll(labels, -time_steps),
        sequence_length=time_steps,
        sequence_stride=1,
        shuffle=shuffle,
        batch_size=batch_size,
    )
    return dataset

# Define metrics
METRICS = [
      ks.metrics.MeanSquaredError(name='MSE'),
      ks.metrics.TruePositives(name='tp'),
      ks.metrics.FalsePositives(name='fp'),
      ks.metrics.TrueNegatives(name='tn'),
      ks.metrics.FalseNegatives(name='fn'),
      ks.metrics.BinaryAccuracy(name='accuracy'),
      ks.metrics.Precision(name='precision'),
      ks.metrics.Recall(name='recall'),
      ks.metrics.AUC(name='auc'),
      ks.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
train_data, val_data, test_data = split_data_chronologically(dataset)
train_scaled, train_labels, val_scaled, val_labels, test_scaled, test_labels = scale_data(train_data, val_data, test_data)

    # Dataset Creation
train_dataset = create_tf_dataset(train_scaled, train_labels, time_steps=time_steps, batch_size=batch_size, shuffle=False)
val_dataset = create_tf_dataset(val_scaled, val_labels, time_steps=time_steps, batch_size=batch_size, shuffle=False)
test_dataset = create_tf_dataset(test_scaled, test_labels, time_steps=time_steps, batch_size=batch_size, shuffle=False)
input_shape = (time_steps, train_scaled.shape[-1])

In [ ]:
# Define the LSTM model with adjustments
model = Sequential([
    LSTM(500, return_sequences=True, input_shape=input_shape, kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),

    LSTM(400, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),

    LSTM(300, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),

    LSTM(200, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),

    LSTM(100, kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))
])

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Define learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_callback = LearningRateScheduler(lr_scheduler)

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping and learning rate scheduler
history = model.fit(train_dataset, epochs=50,batch_size=128,
                    validation_data=val_dataset, class_weight = class_weight)

# Evaluate the model
loss, accuracy,recall, precision = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test recall:", recall)
print("Test precision:", precision)

# Make predictions
predictions = model.predict(test_dataset)



Epoch 1/50
16905/16905 [==============================] - 1471s 84ms/step - loss: 1.8739 - accuracy: 0.6210 - recall_1: 0.6621 - precision_1: 0.4301 - val_loss: 0.5954 - val_accuracy: 0.6983 - val_recall_1: 0.6045 - val_precision_1: 0.4348
Epoch 2/50
16905/16905 [==============================] - 1408s 83ms/step - loss: 0.6158 - accuracy: 0.6553 - recall_1: 0.7079 - precision_1: 0.4655 - val_loss: 0.5732 - val_accuracy: 0.6770 - val_recall_1: 0.6467 - val_precision_1: 0.4149
Epoch 3/50
16905/16905 [==============================] - 1413s 84ms/step - loss: 0.6087 - accuracy: 0.6590 - recall_1: 0.7102 - precision_1: 0.4692 - val_loss: 0.5896 - val_accuracy: 0.6621 - val_recall_1: 0.7010 - val_precision_1: 0.4062
Epoch 4/50
16905/16905 [==============================] - 1414s 84ms/step - loss: 0.6058 - accuracy: 0.6613 - recall_1: 0.7114 - precision_1: 0.4716 - val_loss: 0.5734 - val_accuracy: 0.6720 - val_recall_1: 0.6833 - val_precision_1: 0.4139
Epoch 5/50
16905/16905 [================

In [ ]:
from tensorflow.keras.models import load_model
import pickle

# Assuming you have trained your model and stored the training history in a variable called 'history'

# Save the trained model
model.save("your_model.keras")

# Save the training history
with open('history.pkl', 'wb') as file:
    pickle.dump(history.history, file)